In [9]:
from PlateObjects import Block, SourcePlateCompound, AssayPlate
from EchoXMLGenerator import Generate, WellIDtoNumber

import pandas as pd
import dexpy
import dexpy.optimal
from dexpy.model import ModelOrder
from sklearn.preprocessing import MinMaxScaler
import string


def MakeDesign(additives, factors):
    # Design size is the factors plus one additive
    design = dexpy.optimal.build_optimal(len(factors)+1,order=ModelOrder.quadratic)
    cols = list(factors)
    cols.append(additives)
    design.columns = cols
    return design

def NormalizeDesign(design,factors):
    for i in design:
        # new scaler for each column
        low = factors[i][0]
        high = factors[i][1]
        scaler = MinMaxScaler(feature_range = (low, high))
        design[i] = scaler.fit_transform(design[[i]])
    design.fillna(0,inplace=True)
    return design



Factors = {'Protein Vol':[20,30],'Protein Conc':[10,20]}
Additives = {'BSA':[0.1,0.5],'Triton':[0.01,0.1]}


design = pd.DataFrame([],columns=list(Factors) + list(Additives))
design = design.append(MakeDesign('BSA', Factors),sort = True)
design.dropna(inplace=True,axis=1)
Factors.update(Additives)
design = NormalizeDesign(design, Factors)
assayplate1 = AssayPlate()

for i in design.index:
    row = design.loc[i,:]
    additive = 'BSA'
    additive += ' '+str(round(row[additive],2))

    for repeat in range(2):
        block = Block(row['Protein Vol'],row['Protein Conc'], 'Arachadionic acid',additive)
        assayplate1.AddBlocks(block) 
assayplate1.MakeTransfer()
transferplan = assayplate1.TransferPlan

#design.to_csv('BSA_Dopt.csv')
design=pd.read_csv('BSA_Dopt.csv')
design[['BSA','Protein Conc']].drop_duplicates()

,BSA,Protein Conc
0,0.100000,10.000000
1,0.100000,16.363636
2,0.390909,10.000000
3,0.500000,10.000000
4,0.100000,20.000000
5,0.500000,20.000000
6,0.500000,14.545455
8,0.245455,13.636364
9,0.245455,20.000000


In [27]:
def V1(v2,C1,C2):
    return (C2*v2)/C1


def VolToAdd(RowFromExpDesign, TotalVol_ul):
    # columns in the design and keys in the stock concs need to 
    # be in the same order
    BSAConc = 10 # mg/ml
    ProteinConc = 734.262 #um

    volArg = V1(TotalVol_ul,BSAConc,RowFromExpDesign['BSA'])
    VolProt = V1(TotalVol_ul,ProteinConc,RowFromExpDesign['Protein Vol'])
    VolBuffer = TotalVol_ul- (volArg+VolProt)
    output = pd.DataFrame([[volArg,VolProt,VolBuffer]],columns = ['volBSA','VolProt','VolBuffer'])
    return output


pipettingplan = pd.DataFrame([],columns = ['volBSA','VolProt','VolBuffer'])
                             
for i in design.index:
    pipettingplan = pipettingplan.append(VolToAdd(design.loc[i,:], 10_000)) # 10 ml of each


pipettingplan.reset_index(inplace=True,drop=True)
pipettingplan = pipettingplan.round(2)


print('Total vol of prot:',pipettingplan['VolProt'].sum())
print('Total vol of BSA:',pipettingplan['volBSA'].sum())
print('Total vol of Buffer:',pipettingplan['VolBuffer'].sum())

#pipettingplan['VolBuffer']-=5000
#pipettingplan = pipettingplan.sort_values('volTriton').drop_duplicates()
#(pd.Series([5000]*6,index=pipettingplan.index) - pipettingplan['VolProt']).sort_values()
#(volBSA['VolProt']/2).sort_values()
(pipettingplan['VolProt']/2).sort_values()

Total vol of prot: 3342.8600000000006
Total vol of BSA: 3181.8099999999995
Total vol of Buffer: 93475.32


3    136.190
4    136.190
7    136.190
8    136.190
0    160.955
6    173.335
9    179.525
1    204.285
2    204.285
5    204.285
Name: VolProt, dtype: float64

In [28]:
print('Rows:\n')
for i in design.index:
    print(i,(i*2)+1, (i*2)+2)

Rows:

0 1 2
1 3 4
2 5 6
3 7 8
4 9 10
5 11 12
6 13 14
7 15 16
8 17 18
9 19 20


In [31]:
design['Prot Col'] = pd.Series([(i*2)+1 for i in design.index])
design['Blank Col'] = pd.Series([(i*2)+2 for i in design.index])
design

,Unnamed: 0,BSA,Protein Conc,Protein Vol,Prot Col,Blank Col
0,0,0.100000,10.000000,23.636364,1,2
1,1,0.100000,16.363636,30.000000,3,4
2,2,0.390909,10.000000,30.000000,5,6
3,3,0.500000,10.000000,20.000000,7,8
4,4,0.100000,20.000000,20.000000,9,10
5,5,0.500000,20.000000,30.000000,11,12
6,6,0.500000,14.545455,25.454545,13,14
7,7,0.500000,20.000000,20.000000,15,16
8,8,0.245455,13.636364,20.000000,17,18
9,9,0.245455,20.000000,26.363636,19,20
